In [55]:
import re
import sys
import codecs
def myfun(input_file):
    p1 = re.compile(r'-\{.*?(zh-hans|zh-cn):([^;]*?)(;.*?)?\}-')
    p2 = re.compile(r'[（\(][，；。？！\s]*[）\)]')
    p3 = re.compile(r'[「『]')
    p4 = re.compile(r'[」』]')
    outfile = codecs.open('std_'+input_file, 'w', 'utf-8')
    with codecs.open(input_file, 'r', 'utf-8') as myfile:
        myfile.seek(0)
        for line in myfile:
            line = p1.sub(r'\2', line)
            line = p2.sub(r'', line)
            line = p3.sub(r'“', line)
            line = p4.sub(r'”', line)
            outfile.write(line)
    outfile.close()

In [151]:
from collections import Counter
from functools import reduce
from operator import add,mul

In [36]:
#myfun('wiki_01')

In [74]:
def tokenize(string): 
    return ''.join(re.findall('[\w|\d]+', string))

In [121]:
wiki_content = ['wiki_00','wiki_01']

In [122]:
wiki_content_processed=[]

In [123]:
wiki_countent_counter = []

In [124]:
def get_content_from_wiki(content):
    for wiki_n in content:
        #print(wiki_n)
        outfile = codecs.open(wiki_n, 'r', 'utf-8')
        outfile.seek(0)
        content_n = outfile.read()
        #print(content_n[:1000])
        wiki_content_processed.append(tokenize(content_n))

In [125]:
get_content_from_wiki(content_name)

In [127]:
def get_n_gram_count(content,gram_length):
    return Counter(content[i:i+gram_length] for i in range(len(content)-gram_length))

In [132]:
for content in wiki_content_processed:
    wiki_countent_counter.append(get_n_gram_count(content,1))

In [162]:
def bag_word_model(content,gram_length):
    counter_array=[]
    for content in content:
        counter_array.append(get_n_gram_count(content,gram_length))
    sum=reduce(add,counter_array)
    return sum

In [163]:
one_gram_bag = bag_word_model(wiki_content_processed,1)

In [173]:
two_gram_bag = bag_word_model(wiki_content_processed,2)

## 1-gram

In [169]:
def get_probablity_from_counts(count):
    all_occurance = sum(count.values())
    min_occurance = min(count.values())
    def get_prob(item):
        return count[item]/all_occurance
    return get_prob

get_char_prob = get_probablity_from_counts(one_gram_bag)

def get_prob_of_string(string):
    return reduce(mul,[get_char_prob(e) for e in string ])

In [170]:
pair = """前天晚上吃晚饭的时候
前天晚上吃早饭的时候""".split('\n')

pair2 = """正是一个好看的小猫
真是一个好看的小猫""".split('\n')

pair3 = """我无言以对，简直
我简直无言以对""".split('\n')

pair=[pair,pair2,pair3]

In [171]:
def compare_probabilty_of_pairs(language_model_func,pairs):
    for (p1,p2) in pairs:
        print("{} with probability {}".format(p1,language_model_func(tokenize(p1))))
        print("{} with probability {}".format(p2,language_model_func(tokenize(p2))))

In [172]:
compare_probabilty_of_pairs(get_prob_of_string,pair)

前天晚上吃晚饭的时候 with probability 2.3162692855769543e-33
前天晚上吃早饭的时候 with probability 5.675427023657171e-33
正是一个好看的小猫 with probability 7.709972478020889e-27
真是一个好看的小猫 with probability 2.398281969973941e-27
我无言以对，简直 with probability 2.4200337614488475e-23
我简直无言以对 with probability 2.4200337614488475e-23


## 2-gram

In [183]:
get_pair_prob = get_probablity_from_counts(two_gram_bag)

In [184]:
def get_2_gram_prob(word,prev):
    if get_pair_prob(prev+word)>0:
        return get_pair_prob(prev+word)/get_char_prob(prev)
    else:
        return get_char_prob(word)

def get_2_gram_string_prob(func,string):
    probabilities = []
    for i,c in enumerate(string):
        prev = '<s>' if i==0 else string[i-1]
        probabilities.append(func(c,prev))
    return reduce(mul,probabilities)

In [185]:
def compare_probabilty_of_pairs2(language_model_func,word_prec_func,pairs):
    for (p1,p2) in pairs:
        print("{} with probability {}".format(p1,language_model_func(word_prec_func,tokenize(p1))))
        print("{} with probability {}".format(p2,language_model_func(word_prec_func,tokenize(p2))))

In [186]:
compare_probabilty_of_pairs2(get_2_gram_string_prob,get_2_gram_prob,pair)

前天晚上吃晚饭的时候 with probability 2.4216224148861075e-24
前天晚上吃早饭的时候 with probability 6.516385828119504e-25
正是一个好看的小猫 with probability 3.2892490412094493e-21
真是一个好看的小猫 with probability 3.983452689530549e-22
我无言以对，简直 with probability 1.0826099101827279e-20
我简直无言以对 with probability 1.2717638101101955e-21
